In [ ]:
# Install dependencies as needed:
!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# List the contents of the dataset directory to find the correct file path
print("Listing dataset directory contents:")
!ls -F "/kaggle/input/e-commerce-fraud-detection-dataset/"

# Set the path to the file you'd like to load
# Corrected: Only provide the relative file name within the dataset
file_path = "transactions.csv"

# Load the latest version
df = kagglehub.load_dataset(
 KaggleDatasetAdapter.PANDAS,
  "umuttuygurr/e-commerce-fraud-detection-dataset",
  file_path,
        # Provide any additional arguments like
          # sql_query or pandas_kwargs. See the
            # documenation for more information:
              # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
              )

print("First 5 records:", df.head())

Listing dataset directory contents:
transactions.csv


/tmp/ipython-input-3363733632.py:15: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'e-commerce-fraud-detection-dataset' dataset.
First 5 records:    transaction_id  user_id  account_age_days  total_transactions_user  \
0               1        1               141                       47   
1               2        1               141                       47   
2               3        1               141                       47   
3               4        1               141                       47   
4               5        1               141                       47   

   avg_amount_user  amount country bin_country channel merchant_category  \
0           147.93   84.75      FR          FR     web            travel   
1           147.93  107.90      FR          FR     web            travel   
2           147.93   92.36      FR          FR     app            travel   
3           147.93  112.47      FR          FR     web           fashion   
4           147.93  132.91      FR          US     web       electronics   

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
            print(os.path.join(dirname, filename))

            # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
            # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-fraud-detection-dataset/transactions.csv


In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/e-commerce-fraud-detection-dataset/transactions.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299695 entries, 0 to 299694
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   transaction_id           299695 non-null  int64  
 1   user_id                  299695 non-null  int64  
 2   account_age_days         299695 non-null  int64  
 3   total_transactions_user  299695 non-null  int64  
 4   avg_amount_user          299695 non-null  float64
 5   amount                   299695 non-null  float64
 6   country                  299695 non-null  object 
 7   bin_country              299695 non-null  object 
 8   channel                  299695 non-null  object 
 9   merchant_category        299695 non-null  object 
 10  promo_used               299695 non-null  int64  
 11  avs_match                299695 non-null  int64  
 12  cvv_result               299695 non-null  int64  
 13  three_ds_flag            299695 non-null  int64  
 14  tran

In [ ]:
df.isnull().sum()


,0
transaction_id,0
user_id,0
account_age_days,0
total_transactions_user,0
avg_amount_user,0
amount,0
country,0
bin_country,0
channel,0
merchant_category,0


In [ ]:
if 'transaction_id' in df.columns:
    df = df.drop(columns=['transaction_id'])
else:
    print("Column 'transaction_id' not found in DataFrame.")

In [ ]:
df = df.drop(columns=['user_id'])


In [ ]:
df.head()


,account_age_days,total_transactions_user,avg_amount_user,amount,country,bin_country,channel,merchant_category,promo_used,avs_match,cvv_result,three_ds_flag,transaction_time,shipping_distance_km,is_fraud
0,141,47,147.93,84.75,FR,FR,web,travel,0,1,1,1,2024-01-06T04:09:39Z,370.95,0
1,141,47,147.93,107.90,FR,FR,web,travel,0,0,0,0,2024-01-09T20:13:47Z,149.62,0
2,141,47,147.93,92.36,FR,FR,app,travel,1,1,1,1,2024-01-12T06:20:11Z,164.08,0
3,141,47,147.93,112.47,FR,FR,web,fashion,0,1,1,1,2024-01-15T17:00:04Z,397.40,0
4,141,47,147.93,132.91,FR,US,web,electronics,0,1,1,1,2024-01-17T01:27:31Z,935.28,0


In [ ]:
import pandas as pd
import numpy as np

# 1. Convert the string column to datetime objects
# Pandas automatically handles the "T" and "Z" format
df['transaction_time'] = pd.to_datetime(df['transaction_time'])

# 2. Extract basic numerical features
df['hour'] = df['transaction_time'].dt.hour
df['day_of_week'] = df['transaction_time'].dt.dayofweek
df['month'] = df['transaction_time'].dt.month

# 3. Apply Cyclical Encoding (Crucial for Time)
# This transforms the linear "0-23" hour into a circle, so the model knows
# that 23:00 (11 PM) is very close to 00:00 (Midnight).

# Hour of day (Cycle = 24)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Day of week (Cycle = 7)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# 4. Drop the original timestamp and intermediate columns
df = df.drop(columns=['transaction_time', 'hour', 'day_of_week', 'month'])

# View the encoded data
print(df.head())

   account_age_days  total_transactions_user  avg_amount_user  amount country  \
0               141                       47           147.93   84.75      FR   
1               141                       47           147.93  107.90      FR   
2               141                       47           147.93   92.36      FR   
3               141                       47           147.93  112.47      FR   
4               141                       47           147.93  132.91      FR   

  bin_country channel merchant_category  promo_used  avs_match  cvv_result  \
0          FR     web            travel           0          1           1   
1          FR     web            travel           0          0           0   
2          FR     app            travel           1          1           1   
3          FR     web           fashion           0          1           1   
4          US     web       electronics           0          1           1   

   three_ds_flag  shipping_distance_km  is_f

In [ ]:
categorical_cols = [
    col for col in df.select_dtypes(include='object').columns
]


df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299695 entries, 0 to 299694
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   account_age_days               299695 non-null  int64  
 1   total_transactions_user        299695 non-null  int64  
 2   avg_amount_user                299695 non-null  float64
 3   amount                         299695 non-null  float64
 4   promo_used                     299695 non-null  int64  
 5   avs_match                      299695 non-null  int64  
 6   cvv_result                     299695 non-null  int64  
 7   three_ds_flag                  299695 non-null  int64  
 8   shipping_distance_km           299695 non-null  float64
 9   is_fraud                       299695 non-null  int64  
 10  hour_sin                       299695 non-null  float64
 11  hour_cos                       299695 non-null  float64
 12  day_sin                       

In [ ]:
df['is_fraud'].value_counts()

,count
is_fraud,
0,293083
1,6612


In [ ]:
from sklearn.preprocessing import StandardScaler

# Columns to scale
scale_cols = ['account_age_days', 'total_transactions_user', 'avg_amount_user','amount',  'shipping_distance_km','hour_sin','hour_cos','day_sin','day_cos']

scaler = StandardScaler()

df[scale_cols] = scaler.fit_transform(df[scale_cols])
df.head()

,account_age_days,total_transactions_user,avg_amount_user,amount,promo_used,avs_match,cvv_result,three_ds_flag,shipping_distance_km,is_fraud,...,bin_country_RO,bin_country_TR,bin_country_US,channel_app,channel_web,merchant_category_electronics,merchant_category_fashion,merchant_category_gaming,merchant_category_grocery,merchant_category_travel
0,-1.584794,-0.61464,-0.001063,-0.301100,0,1,1,1,0.032504,0,...,False,False,False,False,True,False,False,False,False,True
1,-1.584794,-0.61464,-0.001063,-0.225674,0,0,0,0,-0.485020,0,...,False,False,False,False,True,False,False,False,False,True
2,-1.584794,-0.61464,-0.001063,-0.276305,1,1,1,1,-0.451209,0,...,False,False,False,True,False,False,False,False,False,True
3,-1.584794,-0.61464,-0.001063,-0.210785,0,1,1,1,0.094350,0,...,False,False,False,False,True,False,True,False,False,False
4,-1.584794,-0.61464,-0.001063,-0.144189,0,1,1,1,1.352045,0,...,False,False,True,False,True,True,False,False,False,False


In [ ]:
X = df.drop(['is_fraud'], axis=1)
y = df['is_fraud']

In [ ]:
from sklearn.model_selection import train_test_split

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Downgrade scikit-learn to a compatible version with imbalanced-learn==0.9.1
# !pip install scikit-learn==1.1.3 # This line caused an installation error, removed.

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42,k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print(f"Original X_train shape: {X_train.shape}")
print(f"Resampled X_train shape: {X_train_res.shape}")
print(f"Original y_train shape: {y_train.shape}")
print(f"Resampled y_train shape: {y_train_res.shape}")

print("\nClass distribution in original y_train:")
print(y_train.value_counts(normalize=True))

print("\nClass distribution in resampled y_train:")
print(y_train_res.value_counts(normalize=True))

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Original X_train shape: (239756, 40)
Resampled X_train shape: (468932, 40)
Original y_train shape: (239756,)
Resampled y_train shape: (468932,)

Class distribution in original y_train:
is_fraud
0    0.977936
1    0.022064
Name: proportion, dtype: float64

Class distribution in resampled y_train:
is_fraud
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(random_state=42,max_iter=1000, class_weight='balanced')
model.fit(X_train_res, y_train_res)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9638966282387094

Confusion Matrix:
 [[56948  1669]
 [  495   827]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     58617
           1       0.33      0.63      0.43      1322

    accuracy                           0.96     59939
   macro avg       0.66      0.80      0.71     59939
weighted avg       0.98      0.96      0.97     59939

